In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [7]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ebeb25e4-2502-49b7-a4c8-d34cc1185b26-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}})

In [8]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='As a large language model, I have no memory of past conversations and therefore do not know your name. You have not told me your name.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e4f19687-1921-4d39-9821-00843a4dbfaa-0', usage_metadata={'input_tokens': 6, 'output_tokens': 30, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})

In [9]:
# 我们发现它没有将之前的对话内容纳入上下文，也无法回答问题。这给聊天机器人带来了糟糕的体验！

# 为了解决这个问题，我们需要将整个对话历史记录传入模型。让我们看看这样做会发生什么：
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. You just told me!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--fb141af4-db42-43b4-ba65-8e27006d54af-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}})

In [10]:
# LangGraph实现了内置持久层，使其成为支持多轮对话的聊天应用程序的理想选择。

# 将我们的聊天模型包装在一个最小的 LangGraph 应用程序中，使我们能够自动保存消息历史记录，从而简化多轮应用程序的开发。

# LangGraph 自带一个简单的内存检查点，我们将在下面使用它。更多详细信息，包括如何使用不同的持久化后端（例如 SQLite 或 Postgres），请参阅其文档。
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

ModuleNotFoundError: No module named 'langgraph'